In [4]:
import json
from landsatxplore.api import API
import getpass
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from hublib.ui import FileUpload
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import FileUpload, Checkbox, FloatSlider, Dropdown, Button, Output, VBox, DatePicker
import urllib.request
from io import BytesIO
from zipfile import ZipFile


In [5]:
import hublib.ui as ui
from hublib.ui import HideCodeButton, RunAllButton

In [6]:
RunAllButton()

Button(description='Run All Cells', style=ButtonStyle())

In [7]:
RunAllButton(label="Run Cells", icon='fa-play-circle-o', style='danger', tooltip='Run All Cells', hide=True)

Button(button_style='danger', description='Run Cells', icon='play-circle-o', style=ButtonStyle(), tooltip='Run…

In [8]:
HideCodeButton()

Button(description='Hide Code Cells', style=ButtonStyle())

In [9]:
# called when all files finish uploading
def done_cb(w, name):
    # Do something with the files here...
    # We just print their names
    print("%s uploaded" % name)
    
    # reset clears and re-enables the widget for more uploads
    # You may want to do this somewhere else, depending on your GUI
    w.reset()

In [10]:
# username_input = widgets.Text(description='Enter your Earth Explorer username: ')
# password_input = widgets.Password(description='Enter your Earth Explorer password: ')
username = input('Enter your Earth Explorer username: ')
password = getpass.getpass('Enter your Earth Explorer password: ')
bbox_file_input = widgets.FileUpload(description='Upload Bbox Shapefile')

Enter your Earth Explorer username: maggiechen259
Enter your Earth Explorer password: ········


In [ ]:
# Process uploaded bbox shapefile
bbox_filename = input('Enter the name of the uploaded bounding box shapefile (with extension): ')
bbox_file_path = f'bbox/{bbox_filename}'

In [ ]:
# # this will allow you to select a single file to be uploaded
# f = FileUpload("Upload Area of Interest", 
#                "This is a description that appears when the mouse hovers over the name.",
#                dir='tmpdir',
#                cb=done_cb,
#                maxsize='20M')
# f
# File upload widget

file_upload = FileUpload(
    description="Upload Shapefile",
    accept=".shp",
    multiple=False
)
file_upload

In [ ]:
start_date = DatePicker(
    description='Start Date',
    disabled = False,
    layout={'width': '300px'}
)
start_date

In [ ]:
end_date = DatePicker(
    description='End Date',
    disabled = False,
    layout={'width': '300px'}
)
end_date

In [ ]:
# Create multiple checkboxes
check1 = ui.Checkbox('Landsat 5', desc='Select Landsat 5', value=False)
check2 = ui.Checkbox('Landsat 7', desc='Select Landsat 7', value=False)
check3 = ui.Checkbox('Landsat 8', desc='Select Landsat 8', value=False)
check4 = ui.Checkbox('Landsat 9', desc='Select Landsat 9', value=False)

In [ ]:
checkboxes_horizontal = widgets.VBox([check1, check2, check3,check4])

# Display the checkboxes in a horizontal layout
display(checkboxes_horizontal)

In [ ]:
# Cloud cover
cloud_cover = ui.Number(
        name='Cloud Cover',
        description="Cloud Cover",
        units='%',
        min=0,
        max=500,
        value=0
)

# cloud_cover = ui.Number(name='Cloud Cover',value=0, min=0, max=100, step=1, description='Cloud Cover (%)')
# cloud_cover = FloatSlider(value=0, min=0, max=100, step=1, description='Cloud Cover (%)')
cloud_cover

In [32]:
# composite color widgets
composite_color = ui.Dropdown(
            name='Composite Color',
            description="Choose the composite color",
            value='True Color',
            options={'True Color', 'False Color'},
#             cb=plot
        )
composite_color

# composite_color = Dropdown(options=['True Color', 'False Color'], value='True Color', description='Composite Color')

Dropdown(children=(HTML(value='<div data-toggle="popover" title="Choose the composite color" data-container="b…

In [33]:
# Output widget to display the result
result_output = Output()

In [34]:
# Button to trigger the search and display results
search_button = Button(description='Display')

In [35]:
# Function to handle button click event
def on_search_button_click(b):
    # Retrieve the file path from the FileUpload widget
    uploaded_files = file_upload.value
    if uploaded_files:
        shapefile_path = list(uploaded_files.keys())[0]
        
        # Read the uploaded shapefile
        bbox = gpd.read_file(shapefile_path)
        bounds = bbox.bounds.iloc[0].to_list()
        
        # Retrieve other widget values
        landsat_options = [check1.value, check2.value, check3.value, check4.value]
        cloud_cover_value = cloud_cover.value
        composite_color_value = composite_color.value
        
        # Perform further processing based on the retrieved values
        with result_output:
            print(f"Shapefile Path: {shapefile_path}")
            print(f"Landsat Options: {landsat_options}")
            print(f"Cloud Cover: {cloud_cover_value}%")
            print(f"Composite Color: {composite_color_value}")

In [36]:
# Attach the function to the button click event
search_button.on_click(on_search_button_click)

In [37]:
# Display the widgets
VBox([file_upload,
      start_date,
      end_date,
      VBox([check1, check2, check3, check4]),
      cloud_cover, 
      composite_color, 
      search_button,
      result_output])


In [38]:
# exe_button = widgets.Button(
#     description = 'Run Analysis',
#     disabled = False,
#     button_style = 'success'
# )
# exe_button.on_click(execute)

In [40]:
#upload area of interest

bbox = gpd.read_file(bbox_file_path)

bounds = bbox.bounds.iloc[0].to_list()

NameError: name 'filename' is not defined

In [ ]:
#Initialize a new API instance
api = API(username, password)

# Perform a request
response = api.request(endpoint="dataset-catalogs")
print(response)

# Search for Landsat TM scenes
scenes = api.search(
    dataset='landsat_ot_c2_l2',
    bbox = bounds,
    start_date= start_date.strftime('%Y-%m-%d'),
    end_date= end_date.strftime('%Y-%m-%d'),
    max_cloud_cover= cloud_cover.value
)
print(f"{len(scenes)} scenes found.")

# Process the result
for scene in scenes:
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))
    # Write scene footprints to disk
    fname = f"{scene['landsat_product_id']}.geojson"
    with open(fname, "w") as f:
        json.dump(scene['spatial_coverage'].__geo_interface__, f)

# log out
api.logout()



In [ ]:
# Create a DataFrame from the scenes
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date']]
df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
df_scenes

In [ ]:
from landsatxplore.earthexplorer import EarthExplorer
import os
import time
import requests


# Initialize the API
api = EarthExplorer(username, password)

# Select the first scene
# Assuming the first scene ID is stored in the variable first_scene_id
ID = df_scenes['display_id'].iloc[0]
output_dir = './data'


# Get the Earth Explorer website URL for the scene
# api_scene_url = f'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&scene_id={ID}&node=INVSVC'


# Open the scene URL in a web browser for manual download
# print(f'You can manually download the scene from the following URL:')
# print(api_scene_url)

# Define the local file path
local_file_path = os.path.join(output_dir, f'{ID}.tar')



# Download the scene using requests
try:
    response = requests.get(api_scene_url)
    if response.status_code == 200:
        with open(local_file_path, 'wb') as f:
            f.write(response.content)
        print(f'{ID} successfully downloaded.')
    else:
        print(f'{ID} download failed. HTTP status code: {response.status_code}')
except Exception as e:
    print(f'{ID} download failed. Error: {str(e)}')

api.logout()


In [ ]:
import tarfile

ID = 'LC08_L2SP_010010_20221004_20221012_02_T1'
file_path = './data/{}.tar'.format(ID)

try:
    tar = tarfile.open(file_path)
    tar.extractall('./data/{}'.format(ID))
    tar.close()
    print('Extraction successful')
except Exception as e:
    print(f'Error: {e}')


In [ ]:
# import tarfile

# # Extract files from tar archive
# tar = tarfile.open('./data/{}.tar'.format(ID))
# tar.extractall('./data/{}'.format(ID))
# tar.close()